Experiments

In [ ]:
import math
import pandas as pd
import numpy as np
import flammkuchen as fl

import sys, os, os.path

sys.path.append(os.path.expanduser("../src"))

from spinorama.plot import plot_params_default, plot_graph, plot_spinorama
from spinorama.load import filter_graphs, graph_melt

df_all = fl.load("../cache.parse_all_speakers.h5")

In [ ]:
# df_klippel = df_all["K"]["Kanto YU6"]["ASR"]["asr"]
df_klippel = df_all["G"]["Genelec 8341A"]["ASR"]["asr-vertical"]

In [ ]:
spin = df_klippel["CEA2034_unmelted"]
spin.keys()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

pio.templates.default = "plotly_white"
width = 600

colors = [
    "#5c77a5",
    "#dc842a",
    "#c85857",
    "#89b5b1",
    "#71a152",
    "#bab0ac",
    "#e15759",
    "#b07aa1",
    "#76b7b2",
    "#ff9da7",
]

uniform_colors = {
    # regression
    "Linear Regression": colors[0],
    "Band ±1.5dB": colors[1],
    "Band ±3dB": colors[1],
    # PIR
    "Estimated In-Room Response": colors[0],
    # spin
    "On Axis": colors[0],
    "Listening Window": colors[1],
    "Early Reflections": colors[2],
    "Sound Power": colors[3],
    "Early Reflections DI": colors[4],
    "Sound Power DI": colors[5],
    # reflections
    "Ceiling Bounce": colors[1],
    "Floor Bounce": colors[2],
    "Front Wall Bounce": colors[3],
    "Rear Wall Bounce": colors[4],
    "Side Wall Bounce": colors[5],
    #
    "Ceiling Reflection": colors[1],
    "Floor Reflection": colors[2],
    #
    "Front": colors[1],
    "Rear": colors[2],
    "Side": colors[3],
    #
    "Total Early Reflection": colors[7],
    "Total Horizontal Reflection": colors[8],
    "Total Vertical Reflection": colors[9],
    # SPL
    "10°": colors[1],
    "20°": colors[2],
    "30°": colors[3],
    "40°": colors[4],
    "50°": colors[5],
    "60°": colors[6],
    "70°": colors[7],
    #
    "500 Hz": colors[1],
    "1000 Hz": colors[2],
    "2000 Hz": colors[3],
    "10000 Hz": colors[4],
    "15000 Hz": colors[5],
}

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
for measurement in ("On Axis", "Listening Window", "Early Reflections", "Sound Power"):
    fig.add_trace(
        go.Scatter(
            x=spin.Freq,
            y=spin[measurement],
            name=measurement,
            # legendgroup="measurements",
            # legendgrouptitle_text="Measurements",
            marker_color=uniform_colors.get(measurement, "black"),
        ),
        secondary_y=False,
    )
for measurement in ("Early Reflections DI", "Sound Power DI"):
    fig.add_trace(
        go.Scatter(
            x=spin.Freq,
            y=spin[measurement],
            name=measurement,
            # legendgroup="DI",
            # legendgrouptitle_text="Directivity",
            marker_color=uniform_colors.get(measurement, "black"),
        ),
        secondary_y=True,
    )

fig.update_xaxes(
    title_text="Frequency (Hz)",
    type="log",
    range=[math.log10(20), math.log10(20000)],
    showline=True,
)
fig.update_yaxes(
    title_text="SPL (dB)",
    secondary_y=False,
    range=[-40, 10],
    dtick=1,
    tickvals=[i for i in range(-40, 10)],
    ticktext=["{}".format(i) if not i % 5 else " " for i in range(-40, 10)],
    showline=True,
)
fig.update_yaxes(
    title_text="DI (dB)                                                    &nbsp;",
    secondary_y=True,
    range=[-5, 45],
    dtick=5,
    tickvals=[-5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45],
    ticktext=["-5", "0", "5", "10", "15", " ", " ", " ", " ", " ", " "],
    showline=True,
)
fig.update_layout(
    title=dict(
        text="Spin for Genelec 8361A from ASR",
        x=0,
        y=0.98,
        xanchor="left",
        yanchor="top",
    ),
    width=600 * math.sqrt(2),
    height=600,
    legend=dict(
        orientation="h",
        x=0,
        y=1,
    ),
    margin={
        "t": 30,
        "b": 0,
        "l": 0,
        "r": 0,
    },
)
fig.show()

In [ ]:
print(df_klippel.keys())
splH = df_klippel["SPL Horizontal_unmelted"]
splV = df_klippel["SPL Vertical_unmelted"]
n_splH = df_klippel["SPL Horizontal_normalized_unmelted"]
n_splV = df_klippel["SPL Vertical_normalized_unmelted"]

In [ ]:
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "Horizontal SPL",
        "Normalized Horizontal SPL",
        "Vertical SPL",
        "Normalized Vertical SPL",
    ),
    shared_xaxes=True,
    shared_yaxes=True,
    horizontal_spacing=0.02,
    vertical_spacing=0.02,
)
for spl, title, row, col in (
    (splH, "Horizontal SPL", 1, 1),
    (n_splH, "Normalized Horizontal SPL", 1, 2),
    (splV, "Vertical SPL", 2, 1),
    (n_splV, "Normalized Vertical SPL", 2, 2),
):
    for measurement in ("On Axis", "10°", "20°", "30°", "40°", "50°", "60°", "70°"):
        fig.add_trace(
            go.Scatter(
                x=spl.Freq,
                y=spl[measurement],
                name=measurement,
                legendgroup="measurements",
                legendgrouptitle_text="Measurements",
                marker_color=uniform_colors.get(measurement, "black"),
                showlegend=(row == 1 and col == 1),
            ),
            row=row,
            col=col,
        )

fig.update_layout(
    title_text="Horizontal & Vertical SPL",
    width=900 * math.sqrt(2),
    height=900,
)
fig.update_xaxes(
    row=2,
    title_text="Frequency (Hz)",
)
fig.update_xaxes(type="log", range=[math.log10(20), math.log10(20000)])
fig.update_yaxes(col=1, title_text="dB", range=[-40, 10], dtick=5)
fig.show()

In [ ]:
from spinorama.compute_misc import compute_contour

min_freq = 200
freq_min = min_freq

In [ ]:
contour_start = -30
contour_end = 3

contour_colorscale = [
    [0, "rgb(0,0,168)"],
    [0.1, "rgb(0,0,200)"],
    [0.2, "rgb(0,74,255)"],
    [0.3, "rgb(0,152,255)"],
    [0.4, "rgb(74,255,161)"],
    [0.5, "rgb(161,255,74)"],
    [0.6, "rgb(255,255,0)"],
    [0.7, "rgb(234,159,0)"],
    [0.8, "rgb(255,74,0)"],
    [0.9, "rgb(222,74,0)"],
    [1, "rgb(253,14,13)"],
]

In [ ]:
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "Horizontal SPL",
        "Normalized Horizontal SPL",
        "Vertical SPL",
        "Normalized Vertical SPL",
    ),
    shared_xaxes=True,
    shared_yaxes=True,
    horizontal_spacing=0.03,
    vertical_spacing=0.03,
)


def plot_contour(spl):
    af, am, az = compute_contour(spl.loc[spl.Freq > freq_min], freq_min)
    az = np.clip(az, contour_start, contour_end)
    return go.Contour(
        x=af[0],
        y=am.T[0],
        z=az,
        contours=dict(
            coloring="fill",
            start=contour_start,
            end=contour_end,
            size=3,
            showlines=False,
        ),
        colorbar=dict(
            dtick=3,
            len=1.0,
            lenmode="fraction",
        ),
        autocolorscale=False,
        colorscale=contour_colorscale,
        opacity=1.0,
    )


for spl, row, col in ((splH, 1, 1), (n_splH, 1, 2), (splV, 2, 1), (n_splV, 2, 2)):
    fig.add_trace(
        plot_contour(spl),
        row=row,
        col=col,
    )

fig.update_layout(title_text="SPL", width=600 * 2, height=800)
fig.update_xaxes(
    type="log",
    range=[math.log10(min_freq), math.log10(20000)],
    layer="above traces",
    showgrid=True,
)
fig.update_xaxes(title_text="Frequency (Hz)", row=2)
fig.update_yaxes(
    title_text="Angle",
    range=[-180, 180],
    dtick=30,
    tickvals=[v for v in range(-180, 210, 30)],
    ticktext=["{}°".format(v) for v in range(-180, 210, 30)],
    col=1,
    showgrid=True,
    gridcolor="white",
    gridwidth=2,
    layer="above traces",
    zeroline=True,
    zerolinewidth=3,
)
fig.layout["grid"] = {
    "columns": 2,
    "rows": 2,
}
fig.update_shapes(
    line={
        "color": "black",
        "width": 3,
    },
    type="line",
    visible=True,
    xsizemode="scaled",
    x0=0,
    x1=1,
    ysizemode="scaled",
    y0=0,
    y1=0,
)
fig.show()

In [ ]:
from spinorama.graph_radar import plot as radar_plot, find_nearest_freq, join, label
from spinorama.load_misc import sort_angles

anglelist = [a for a in range(-180, 180, 10)]

In [ ]:
def projection(anglelist, gridZ, hz):
    # map in 2d
    dbsR = [db for a, db in zip(anglelist, gridZ)]
    dbsTheta = [a for a, db in zip(anglelist, gridZ)]

    # join with first point (-180=180)
    # print('Calling project', anglelist)
    dbsR.append(dbsR[0])
    dbsTheta.append(dbsTheta[0])

    return dbsR, dbsTheta, [hz for i in range(0, len(dbsR))]


def plot(anglelist, df):
    dfu = sort_angles(df)
    db_mean = np.mean(dfu.loc[(dfu.Freq > 900) & (dfu.Freq < 1100)]["On Axis"].values)
    freq = dfu.Freq
    dfu = dfu.drop("Freq", axis=1)
    db_min = np.min(dfu.min(axis=0).values)
    db_max = np.max(dfu.max(axis=0).values)
    db_scale = max(abs(db_max), abs(db_min))
    # if df is normalized then 0 will be at the center of the radar which is not what
    # we want. Let's shift the whole graph up.
    # if db_mean < 45:
    #    dfu += db_scale
    # print(db_min, db_max, db_mean, db_scale)
    # build 3 plots
    dbX = []
    dbY = []
    hzZ = []
    for hz in [500, 1000, 2000, 10000, 15000]:
        ihz = find_nearest_freq(freq, hz)
        if ihz is None:
            continue
        X, Y, Z = projection(anglelist, dfu.loc[ihz][dfu.columns != "Freq"], hz)
        # add to global variable
        dbX.append(X)
        dbY.append(Y)
        hzZ.append(Z)

    # normalise
    dbX = [v2 for v1 in dbX for v2 in v1]
    dbY = [v2 for v1 in dbY for v2 in v1]
    # print("dbX min={} max={}".format(np.array(dbX).min(), np.array(dbX).max()))
    # print("dbY min={} max={}".format(np.array(dbY).min(), np.array(dbY).max()))

    hzZ = [label(i2) for i1 in hzZ for i2 in i1]

    return db_mean, pd.DataFrame({"R": dbX, "Theta": dbY, "Freq": hzZ})

In [ ]:
_, dbs_df_H = plot(anglelist, splH)
_, dbs_df_V = plot(anglelist, splV)

In [ ]:
# px.line_polar(dbs_df_H, r="R", theta="Theta", color="Freq", width=500, height=500)
px.line_polar(dbs_df_V, r="R", theta="Theta", color="Freq", width=500, height=500)
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Horizontal SPL", "Vertical SPL"),
    specs=[[{"type": "polar"}, {"type": "polar"}]],
)

for spl, col in ((dbs_df_H, 1), (dbs_df_V, 2)):
    for freq in np.unique(spl["Freq"].values):
        slice = spl.loc[spl.Freq == freq]
        fig.add_trace(
            go.Scatterpolar(
                r=slice.R,
                theta=slice.Theta,
                dtheta=30,
                name=freq,
                marker_color=uniform_colors.get(freq, "black"),
                legendgroup="measurements",
                legendgrouptitle_text="Frequencies",
                legendrank=int(freq[:-3]),
                showlegend=(col == 1),
            ),
            row=1,
            col=col,
        )
fig.update_layout(
    title_text="SPL",
    width=600 * 2,
    height=800,
    polar=dict(
        radialaxis=dict(
            range=[-45, 5],
            dtick=5,
        ),
        angularaxis=dict(
            dtick=10,
            tickvals=list(range(0, 360, 10)),
            ticktext=[
                "{}°".format(x) if abs(x) < 60 or not x % 30 else " "
                for x in (list(range(0, 190, 10)) + list(range(-170, 0, 10)))
            ],
        ),
    ),
    polar2=dict(
        radialaxis=dict(range=[-45, 5], dtick=5),
        angularaxis=dict(
            dtick=10,
            tickvals=list(range(0, 360, 10)),
            ticktext=[
                "{}°".format(x) if abs(x) < 60 or not x % 30 else " "
                for x in (list(range(0, 190, 10)) + list(range(-170, 0, 10)))
            ],
        ),
    ),
)
fig.show()

In [ ]:
from scipy import stats

pir = df_klippel["Estimated In-Room Response_unmelted"]
pir_restricted = pir.loc[(pir.Freq > 300) & (pir.Freq < 10000)]

slope, intercept, r, p, se = stats.linregress(x=pir_restricted.Freq, y=pir_restricted["Estimated In-Room Response"])

# 600 px = 50 dB
height = 600
one_db = 600 / 50
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=pir.Freq,
        y=[slope * math.log10(f) + intercept for f in pir.Freq],
        line=dict(width=2, color="black"),
        opacity=1,
        name="Linear regression",
    )
)
fig.add_trace(
    go.Scatter(
        x=pir.Freq,
        y=[slope * math.log10(f) + intercept for f in pir.Freq],
        line=dict(width=3 * one_db, color="gray"),
        opacity=0.15,
        name="Band ±1.5dB",
    )
)
fig.add_trace(
    go.Scatter(
        x=pir.Freq,
        y=[slope * math.log10(f) + intercept for f in pir.Freq],
        line=dict(width=6 * one_db, color="gray"),
        opacity=0.1,
        name="Band ±3dB",
    )
)
fig.add_trace(
    go.Scatter(
        x=pir.Freq,
        y=pir["Estimated In-Room Response"],
        name="PIR",
        line=dict(width=3, color=uniform_colors["Estimated In-Room Response"]),
    )
)
fig.update_xaxes(title_text="Frequency (Hz)", type="log", range=[math.log10(20), math.log10(20000)])
fig.update_yaxes(title_text="SPL (dB)", range=[50, 100], dtick=5)
fig.update_layout(width=600 * math.sqrt(2), height=600)
fig.show()

In [ ]:
from spinorama.compute_misc import directivity_matrix

In [ ]:
x, y, z = directivity_matrix(splH, splV)

In [ ]:
fig = go.Figure(
    go.Contour(
        x=x[0],
        y=y.T[0],
        z=z,
        contours=dict(start=-1, end=1, size=0.05),
    )
)
fig.update_layout(width=600, height=600)
fig.show()

In [ ]:
def plot_surface(spl):
    af, am, az = contour(spl.loc[spl.Freq > 20])
    return go.Surface(
        x=af[0],
        y=am.T[0],
        z=az,
        showscale=False,
    )


fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Horizontal SPL", "Vertical SPL"),
    shared_yaxes=True,
    horizontal_spacing=0.02,
    specs=[[{"type": "surface"}, {"type": "surface"}]],
)
fig.add_trace(plot_surface(n_splH), row=1, col=1)
fig.add_trace(plot_surface(n_splV), row=1, col=2)
scene_surface = {
    "xaxis": {"nticks": 20, "type": "log", "title": "Hz"},
    "yaxis": {"dtick": 30, "title": "Angle"},
    "zaxis": {"nticks": 10, "range": [-45, 5], "title": "SPL"},
    # "aspectratio": {"x": 1, "y": 1, "z": 0.5}
}
fig.update_layout(
    width=1200,
    height=1000,
    scene=scene_surface,
    scene2=scene_surface,
)
fig.show()

In [ ]:
fig = go.Figure()


def plot_contour(spl):
    af, am, az = compute_contour(spl.loc[spl.Freq > freq_min], freq_min)
    az = np.clip(az, contour_start, contour_end)
    return go.Contour(
        x=af[0],
        y=am.T[0],
        z=az,
        contours=dict(
            coloring="fill",
            start=contour_start,
            end=contour_end,
            size=3,
            showlines=False,
        ),
        colorbar=dict(
            dtick=3,
            len=1.0,
            lenmode="fraction",
        ),
        autocolorscale=False,
        colorscale=contour_colorscale,
        opacity=1.0,
    )


fig.add_trace(plot_contour(splH))

fig.update_layout(title_text="SPL", width=600 * 2, height=800)
fig.update_xaxes(
    type="log",
    range=[math.log10(200), math.log10(20000)],
)
fig.update_xaxes(title_text="Frequency (Hz)")
fig.update_yaxes(
    title_text="Angle",
    range=[-180, 180],
    dtick=30,
    tickvals=[v for v in range(-180, 210, 30)],
    ticktext=["{}°".format(v) for v in range(-180, 210, 30)],
)


def add_box(lfig, x, y):
    lfig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            opacity=0.5,
            marker_color="white",
            line_width=1,
        )
    )


def compute_horizontal_lines(x_min, x_max, y_data):
    x = np.tile([x_min, x_max, None], len(y_data))
    y = np.ndarray.flatten(np.array([[a, a, None] for a in y_data]))
    return x, y


def compute_vertical_lines(y_min, y_max, x_data):
    y = np.tile([y_min, y_max, None], len(x_data))
    x = np.ndarray.flatten(np.array([[a, a, None] for a in x_data]))
    return x, y


hx, hy = compute_horizontal_lines(200, 20000, range(-150, 180, 30))
vrange = [100 * i for i in range(3, 9)] + [1000 * i for i in range(1, 10)] + [10000 + 1000 * i for i in range(1, 5)]
vx, vy = compute_vertical_lines(-180, 180, vrange)

add_box(fig, hx, hy)
add_box(fig, vx, vy)

fig.show()